Here we take a different approach. We still optimize 4 weeks at a time, but we freeze decisions just for the first week in the interval, later we move the window by one week and repeat the process.

In [33]:
using DataFrames
using JuMP, Gurobi

In [34]:
# number of teams
N = 20
# total number of weeks
W = N-1
# number of days per weeks
D = 7;

In [35]:
function GetC()

    W_ = 19

    C = zeros((D*(W_+1), D*(W_+1)))

    for t in 1:(W_*D)
        for t_ in 1:(W_*D)
            if t < t_
                C[t, t_] = (t_ - t)^(-1)
            end
        end
    end

    return C
end

GetC (generic function with 1 method)

In [36]:
function calendar_week(x,week)
    df = DataFrame(Player=1:N, Tuesday=0.0, Wednesday=0.0, Thursday=0.0, Friday=0.0, Saturday=0.0, Sunday=0.0, Monday=0.0)
    for i=1:N
        for j=1:N
            for d=1:D
                if x[i,j,week,d] == 1.0
                        df[i, 1+d] = j
                end
            end
        end
    end
    return df
end

calendar_week (generic function with 1 method)

In [44]:
function Optimize4WeeksAndFreezeFirst(Y_old, Win, Wend, last_period)

    # create model
    model = Model(Gurobi.Optimizer)

    # set time limit
    set_optimizer_attribute(model, "TimeLimit", 600)

    # dont print output
    set_silent(model)

    # PARAMETERS
    C = GetC()
    N = 20 # teams
    D = 7 # days

    # VARIABLES
    @variable(model, x[i = 1:N, j = 1:N, w = Win:Wend, d = 1:D], Bin) # 1 if team i plays team j on day d of week w, 0 otherwise
    @variable(model, a[i = 1:N, j = 1:N, k = 1:N, w = Win:Wend, d = 1:D, d_ = 1:D] >= 0) # will be pushed to be binary
    @variable(model, y[i = 1:N, j = 1:N], Bin) # 1 if team i played team j on w, 0 otherwise

    # OBJECTIVE FUNCTION
    # put penalty on having y[i,j] = 1, otherwise formulation put all of them to 1
    @objective(model, Min, sum(C[7 * w + d, 7 * (w+1) + d_] * a[i, j, k, w, d, d_] for i in 1:N, 
        j in 1:N, k in 1:N, w in Win:Wend-1, d in 1:D, d_ in 1:D) + sum(y[i, j] for i = 1:N, j = 1:N))

    # CONSTRAINTS
    # linearize objective function
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = Win:Wend-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] <= x[i, j, w, d])
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = Win:Wend-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] <= x[i, k, w+1, d])
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = Win:Wend-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] >= x[i, j, w, d] + x[i, k, w+1, d] - 1)

    # teams can never play against themselves
    @constraint(model, [i = 1:N, w = Win:Wend, d = 1:D], x[i, i, w, d] == 0)

    # if A plays against B, then B plays against A
    @constraint(model, [i = 1:N, j = 1:N, w = Win:Wend, d = 1:D], x[i, j, w, d] == x[j, i, w, d])

    # each team can only play at most one game per week
    @constraint(model, [i = 1:N, w = Win:Wend], sum(x[i, j, w, d] for j in 1:N, d in 1:D) <= 1)

    # each team plays Wend - Win + 1 games in total
    @constraint(model, [i = 1:N], sum(x[i, j, w, d] for j in 1:N, w in Win:Wend, d in 1:D) == Wend - Win + 1)

    # each game is against a different team, team play against each other team at most once
    @constraint(model, [i = 1:N, j = 1:N], sum(x[i, j, w, d] for w in Win:Wend, d in 1:D) <= 1)

    # if team i plays team j, update Y
    @constraint(model, [i = 1:N, j = 1:N], y[i, j] >= sum(x[i, j, w, d] for d in 1:D, w = Win:Wend))

    @constraint(model, [i = 1:N, j = 1:N], y[i, j] == y[j, i])
    
    # if team i played team j in the past, then they can't play again now
    @constraint(model, [i = 1:N, j = 1:N], sum(x[i,j,w,d] for w in Win:Wend, d in 1:D) <= 1 - Y_old[i, j])

    # cannot plays on days 1,2,3
    @constraint(model, [i = 1:N, j = 1:N, w = Win:Wend, d = 1:3], x[i, j, w, d] == 0)

    # OPTIMIZE
    optimize!(model)

    # update old Y before returning
    # differently than previous formulation, we update y only if (i, j) played in the first week of the interval
    if last_period
        # update all Y
        Y_old += value.(y)
    else   
        # update only the first week
        for i = 1:N
            for j = 1:N
                for d = 1:D
                    if value.(x[i, j, Win, d]) == 1
                        Y_old[i, j] += value.(y[i, j])
                    end
                end
            end
        end
    end

    return value.(x), Y_old

end


Optimize4WeeksAndFreezeFirst (generic function with 2 methods)

In [38]:
# define interval
interval = 3    # number of weeks in each interval in addition to first week

# first time, Y_old initialized to empty
Y_old = zeros(20, 20)   # teams never played against each other before

# define calendar list, each time append the calendar week to 
calendar = []
;

In [45]:
for Win in 1:W-interval

    Wend = Win + interval

    println("Initial week: ", Win, ", end week: ", Wend)
    println("#############################################")

    # get value of X and Y
    X, Y_old = Optimize4WeeksAndFreezeFirst(Y_old, Win, Wend, Wend == W)

    if Wend == W
        for w in Win:Wend
            push!(calendar, calendar_week(X, w)) 
        end
    else
        push!(calendar, calendar_week(X[:, :, Win:Win, :], Win))
    end

end

In [41]:
for w in 1:W
    display(calendar[w])
end

LoadError: BoundsError: attempt to access 0-element Vector{Any} at index [1]

In [43]:
# in this matrix all elements except the diagonal should be 1
println(sum([Y_old[i,i] for i in 1:20]))

# all other elements should be 1 (400 elements in total, diagonal = 20 element is 0, sum should be 380)
println(sum(Y_old))

0.0
0.0


In [42]:
[i for (i, el) in enumerate(Y_old[1,:]) if el == 1.0] 

Int64[]